# Trading Strategy Development Project

The goal of our project is to develop an application that allows user to develop, and backtest their own trading strategies, providing them key information about the efficiency and reliability of their strategy

In [4]:
import os
from dotenv import load_dotenv

# Getting backtesting.py to work and implementing their default strategy

# Creating our own strategy

# Papertrading API (Alpaca)

In [ ]:
!pip install alpaca-py

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

alpaca_key = os.getenv('ALPACA_KEY')
secret_key = os.getenv('SECRET_KEY')

In [4]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [ ]:
trading_client = TradingClient(alpaca_key, secret_key, paper=True)

# preparing market order
market_order_data = MarketOrderRequest(
                    symbol="TSLA",
                    qty=5,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

market_order = trading_client.submit_order(
                order_data=market_order_data
               )


In [5]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
# Download data
ticker = 'TSLA'
data = yf.download(ticker, period='10y', interval='1d')



[*********************100%***********************]  1 of 1 completed

1 Failed download:
['TSLA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


In [124]:
(data['RSI_Cross_30'].values & (data['Close'].values.flatten() > data['MA50'].values))

array([False, False, False, ..., False, False, False])

In [120]:
data['RSI_Cross_30'] 

Datetime
2023-02-23 09:30:00    False
2023-02-23 10:30:00    False
2023-02-23 11:30:00    False
2023-02-23 12:30:00    False
2023-02-23 13:30:00    False
                       ...  
2025-02-12 11:30:00    False
2025-02-12 12:30:00    False
2025-02-12 13:30:00    False
2025-02-12 14:30:00    False
2025-02-12 15:30:00    False
Name: RSI_Cross_30, Length: 3445, dtype: bool

In [38]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time

BASE_URL = "https://paper-api.alpaca.markets"

api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')

def get_data(symbol, limit=3630):
    barset = api.get_bars(symbol, timeframe='1Min', limit=limit).df
    df = barset[['close']].copy()
    df['close'] = df['close'].astype(float)
    return df
SYMBOL = 'TSLA'

get_data(SYMBOL)


,close
timestamp,
2025-02-21 09:00:00+00:00,353.7000
2025-02-21 09:01:00+00:00,353.4000
2025-02-21 09:02:00+00:00,353.2800
2025-02-21 09:03:00+00:00,353.3000
2025-02-21 09:04:00+00:00,353.3600
...,...
2025-02-21 17:17:00+00:00,341.6271
2025-02-21 17:18:00+00:00,341.2400
2025-02-21 17:19:00+00:00,340.9700


In [3]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time
import os
from dotenv import load_dotenv
load_dotenv()

alpaca_key = os.getenv('ALPACA_KEY')
secret_key = os.getenv('SECRET_KEY')

# Alpaca API Credentials
BASE_URL = "https://paper-api.alpaca.markets"

# Initialize Alpaca API
api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')

# Trading Parameters
SYMBOL = "TSLA"
SHORT_WINDOW = 10
LONG_WINDOW = 50
RSI_PERIOD = 10
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
ORDER_SIZE = 1


def get_data(symbol, limit=3630):
    barset = api.get_bars(symbol, timeframe='1Min', limit=limit).df
    df = barset[['close']].copy()
    df['close'] = df['close'].astype(float)
    return df


# Function to calculate indicators
def calculate_indicators(df):
    df['SMA_Short'] = df['close'].rolling(window=SHORT_WINDOW).mean()
    df['SMA_Long'] = df['close'].rolling(window=LONG_WINDOW).mean()
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=RSI_PERIOD).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=RSI_PERIOD).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Function to place orders
def place_order(side):
    try:
        api.submit_order(
            symbol=SYMBOL,
            qty=ORDER_SIZE,
            side=side,
            type='market',
            time_in_force='gtc'
        )
        print(f"{side.upper()} order placed for {SYMBOL}")
    except Exception as e:
        print(f"Order failed: {e}")

# Main loop for trading
while True:
    df = get_data(SYMBOL)
    df = calculate_indicators(df)
    latest = df.iloc[-1]
    print(latest['RSI'])

    position = api.get_position(SYMBOL) if SYMBOL in [pos.symbol for pos in api.list_positions()] else None

    if latest['SMA_Short'] > latest['SMA_Long'] and latest['RSI'] < RSI_OVERBOUGHT:
        place_order("buy")
    elif latest['SMA_Short'] < latest['SMA_Long'] and latest['RSI'] > RSI_OVERSOLD and position is not None:
        place_order("sell")
    if latest['RSI'] < RSI_OVERSOLD and position is None:
        place_order("buy")
    elif latest['RSI'] > RSI_OVERBOUGHT and position is not None:
        place_order("sell")
    time.sleep(10)  # Run every minute


75.0
68.00915635470918
BUY order placed for TSLA


KeyboardInterrupt: 

In [ ]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time
import os
from dotenv import load_dotenv
load_dotenv()

alpaca_key = os.getenv('ALPACA_KEY')
secret_key = os.getenv('SECRET_KEY')

import pandas as pd
import time

# Alpaca API Credentials
BASE_URL = "https://paper-api.alpaca.markets"
DATA_URL = "https://data.alpaca.markets/v2"

# Initialize Alpaca API
api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')
data_api = tradeapi.REST(alpaca_key, secret_key, DATA_URL, api_version='v2')

# Trading Parameters
SYMBOL = "SPY"
SHORT_WINDOW = 10
LONG_WINDOW = 50
RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
ORDER_SIZE = 1

def get_historical_data(symbol, start, end):
    try:
        barset = api.get_bars(symbol, timeframe='1Min', start=start, end=end).df
        if barset.empty:
            print("No data fetched for the given symbol and date range.")
            return pd.DataFrame()  # Return an empty DataFrame
        df = barset[['close']].copy()
        df['close'] = df['close'].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame

# Function to calculate indicators
def calculate_indicators(df):
    if df.empty:
        print("Empty DataFrame, skipping indicator calculation.")
        return df
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=RSI_PERIOD).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=RSI_PERIOD).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Function to place orders
def place_order(side):
    try:
        api.submit_order(
            symbol=SYMBOL,
            qty=ORDER_SIZE,
            side=side,
            type='market',
            time_in_force='gtc'
        )
        print(f"{side.upper()} order placed for {SYMBOL}")
    except Exception as e:
        print(f"Order failed: {e}")

# Fetch historical data
start_date = '2020-01-08'
end_date = '2020-01-09'

df = get_historical_data(SYMBOL, start=start_date, end=end_date)
df = calculate_indicators(df)

# Simulate real-time data updates
if not df.empty:
    for i in range(len(df)):
        latest = df.iloc[i]
        print(f"Index: {latest.name}, RSI: {latest['RSI']}")

        position = api.get_position(SYMBOL) if SYMBOL in [pos.symbol for pos in api.list_positions()] else None

        if latest['RSI'] < RSI_OVERSOLD:
            place_order("buy")
        elif latest['RSI'] > RSI_OVERBOUGHT and position is not None:
            place_order("sell")
        time.sleep(1)  # Simulate waiting for the next minute
else:
    print("No data available to simulate real-time updates.")

In [15]:
import alpaca_trade_api as tradeapi
import pandas as pd
import time
import os
from dotenv import load_dotenv
load_dotenv()

alpaca_key = os.getenv('ALPACA_KEY')
secret_key = os.getenv('SECRET_KEY')

import pandas as pd
import time

# Alpaca API Credentials
BASE_URL = "https://paper-api.alpaca.markets"
DATA_URL = "https://data.alpaca.markets/v2"

# Initialize Alpaca API
api = tradeapi.REST(alpaca_key, secret_key, BASE_URL, api_version='v2')
data_api = tradeapi.REST(alpaca_key, secret_key, DATA_URL, api_version='v2')

# Trading Parameters
SYMBOL = "SPY"
SHORT_WINDOW = 10
LONG_WINDOW = 50
RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
ORDER_SIZE = 1

def get_historical_data(symbol, start, end):
    try:
        barset = api.get_bars(symbol, timeframe='1Min', start=start, end=end).df
        if barset.empty:
            print("No data fetched for the given symbol and date range.")
            return pd.DataFrame()  # Return an empty DataFrame
        df = barset[['close']].copy()
        df['close'] = df['close'].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame

In [10]:
import sqlite3
conn = sqlite3.connect("C:\\Users\\ryanl\\Documents\\Coursework_Python\\PIC16B Trading Project\\strategy_db.sqlite")
cursor = conn.cursor()
        
        # Get all strategies
cursor.execute("SELECT * FROM strategy")
strategies = cursor.fetchall()
print(strategies)



[('27f0e213', "Jeffrey's Strategy", 2.0, '[{"type": "RSI", "settings": {"period": 14, "overbought": 70, "oversold": 35}}, {"type": "BBANDS", "settings": {"period": 20, "std_dev": 2}}]')]


In [11]:
import pandas_ta as ta
import json
# Read indicators from our data frame and their parameters

conn = sqlite3.connect("C:\\Users\\ryanl\\Documents\\Coursework_Python\\PIC16B Trading Project\\strategy_db.sqlite")
cursor = conn.cursor()
        
        # Get all strategies
cursor.execute("SELECT * FROM strategy")
strategies = cursor.fetchall()

#[('f298e14a', 'Ryan', 1.0, '[{"type": "MACD", "settings": {"fast_period": 12, "slow_period": 26, "signal_period": 9}}, {"type": "BBANDS", "settings": {"period": 20, "std_dev": 2}}]')]
# Find the indicators in the strategy
indicators = strategies[0][3]
#Extract each indicator and the settings from the json string for each indicator
indicators = json.loads(indicators)
sma_settings = []
rsi_settings = []
stoch_settings = []
smi_settings = []
macd_settings = []
bbands_settings = []

for indicator in indicators:
    indicator_type = indicator['type']
    indicator_settings = indicator['settings']
    print(indicator_type)
    if indicator_type == 'SMA':
        period = indicator_settings['period']
        sma_settings.append(period)
        print(period)
    elif indicator_type == 'RSI':
        period = indicator_settings['period']
        overbought = indicator_settings['overbought']
        oversold = indicator_settings['oversold']
        rsi_settings.append(period)
        rsi_settings.append(overbought)
        rsi_settings.append(oversold)
        print(period)
        print(overbought)
        print(oversold)
    elif indicator_type == 'STOCH':
        k_period = indicator_settings['k_period']
        d_period = indicator_settings['d_period']
        stoch_settings.append(k_period)
        stoch_settings.append(d_period)
        print(k_period)
        print(d_period)
    elif indicator_type == 'SMI':
        period = indicator_settings['period']
        signal_period = indicator_settings['signal_period']
        smi_settings.append(period)
        smi_settings.append(signal_period)
        print(period)
        print(signal_period)
    elif indicator_type == 'MACD':
        fast_period = indicator_settings['fast_period']
        slow_period = indicator_settings['slow_period']
        signal_period = indicator_settings['signal_period']
        macd_settings.append(fast_period)
        macd_settings.append(slow_period)
        macd_settings.append(signal_period)
        print(fast_period)
        print(slow_period)
        print(signal_period)
    elif indicator_type == 'BBANDS':
        period = indicator_settings['period']
        std_dev = indicator_settings['std_dev']
        bbands_settings.append(period)
        bbands_settings.append(std_dev)
        print(period)
        print(std_dev)
    else:
        print("Indicator not recognized")

RSI
14
70
35
BBANDS
20
2


In [14]:
import pandas as pd
import pandas_ta as ta
import time

# Function to calculate indicators
def calculate_indicators(df, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings):
    if df.empty:
        print("Empty DataFrame, skipping indicator calculation.")
        return df
        
    # Create a copy to avoid modifying the original
    result_df = df.copy()
    
    # SMA calculation
    if sma_settings and len(sma_settings) >= 2:
        result_df['SMA_fast'] = ta.sma(result_df['close'], length=sma_settings[0])
        result_df['SMA_slow'] = ta.sma(result_df['close'], length=sma_settings[1])
    
    # RSI calculation
    if rsi_settings and len(rsi_settings) >= 1:
        result_df['RSI'] = ta.rsi(result_df['close'], length=rsi_settings[0])
    
    # Stochastic calculation
    if stoch_settings and len(stoch_settings) >= 2:
        stoch_df = ta.stoch(
            result_df['high'], 
            result_df['low'], 
            result_df['close'], 
            k=stoch_settings[0], 
            d=stoch_settings[1], 
            smooth_k=stoch_settings[2] if len(stoch_settings) > 2 else 3
        )
        # Add stochastic columns to the main dataframe
        for col in stoch_df.columns:
            result_df[col] = stoch_df[col]
    
    # SMI calculation
    if smi_settings and len(smi_settings) >= 2:
        smi_df = ta.smi(
            result_df['close'], 
            length=smi_settings[0], 
            signal=smi_settings[1]
        )
        # Add SMI columns to the main dataframe
        for col in smi_df.columns:
            result_df[col] = smi_df[col]
    
    # MACD calculation
    if macd_settings and len(macd_settings) >= 3:
        fast, slow, signal = macd_settings
        macd_df = ta.macd(
            result_df['close'], 
            fast=fast, 
            slow=slow, 
            signal=signal
        )
        # Add MACD columns to the main dataframe
        for col in macd_df.columns:
            result_df[col] = macd_df[col]
            
        # Store column names for easier reference
        result_df['macd_line_col'] = macd_df.columns[0]
        result_df['macd_signal_col'] = macd_df.columns[1]
        result_df['macd_hist_col'] = macd_df.columns[2] if len(macd_df.columns) > 2 else None
    
    # Bollinger Bands calculation
    if bbands_settings and len(bbands_settings) >= 2:
        bbands_df = ta.bbands(
            result_df['close'], 
            length=bbands_settings[0], 
            std=bbands_settings[1]
        )
        # Add BBands columns to the main dataframe
        for col in bbands_df.columns:
            result_df[col] = bbands_df[col]
            
        # Store column names for easier reference
        result_df['bbands_upper_col'] = bbands_df.columns[0]
        result_df['bbands_mid_col'] = bbands_df.columns[1]
        result_df['bbands_lower_col'] = bbands_df.columns[2]
    
    return result_df

# Check buy conditions for a SINGLE row of data
def check_buy_conditions_for_row(row, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings):
    """
    Check buy conditions for a single row of data
    Returns True if all conditions are met, False otherwise
    """
    conditions_met = []
    conditions_total = 0
    
    # SMA condition: fast > slow
    if sma_settings and len(sma_settings) >= 2:
        conditions_total += 1
        if 'SMA_fast' in row and 'SMA_slow' in row:
            sma_condition = row['SMA_fast'] > row['SMA_slow']
            conditions_met.append(sma_condition)
    
    # RSI condition: oversold (RSI < threshold)
    if rsi_settings and len(rsi_settings) >= 3:
        conditions_total += 1
        if 'RSI' in row:
            rsi_condition = row['RSI'] < rsi_settings[2]
            conditions_met.append(rsi_condition)
            if rsi_condition:
                print('RSI Met at RSI: ', row['RSI'])
    
    # Stochastic condition
    if stoch_settings and len(stoch_settings) >= 2:
        conditions_total += 1
        k_period = stoch_settings[0]
        d_period = stoch_settings[1]
        smooth_k = stoch_settings[2] if len(stoch_settings) > 2 else 3
        
        # Typical column names
        k_col = f"STOCHk_{k_period}_{d_period}_{smooth_k}"
        d_col = f"STOCHd_{k_period}_{d_period}_{smooth_k}"
        
        if k_col in row and d_col in row:
            stoch_condition = (row[k_col] > row[d_col]) and (row[k_col] < 20) and (row[d_col] < 20)
            conditions_met.append(stoch_condition)
    
    # SMI condition
    if smi_settings and len(smi_settings) >= 2:
        conditions_total += 1
        length = smi_settings[0]
        signal_length = smi_settings[1]
        
        # Typical column names
        smi_col = f"SMI_{length}_{signal_length}"
        smi_signal_col = f"SMIs_{length}_{signal_length}"
        
        if smi_col in row and smi_signal_col in row:
            smi_condition = row[smi_col] > 0
            conditions_met.append(smi_condition)
    
    # MACD condition: MACD line > Signal line
    if macd_settings and len(macd_settings) >= 3:
        conditions_total += 1
        
        if 'macd_line_col' in row and 'macd_signal_col' in row:
            macd_col = row['macd_line_col']
            signal_col = row['macd_signal_col']
            
            if macd_col in row and signal_col in row:
                macd_condition = row[macd_col] > row[signal_col]
                conditions_met.append(macd_condition)
                if macd_condition:
                    print('MACD met at', row[macd_col], ' ', row[signal_col])

    
    # Bollinger Bands condition: price < lower band
    if bbands_settings and len(bbands_settings) >= 2:
        conditions_total += 1
        
        if 'bbands_lower_col' in row:
            lower_band_col = row['bbands_lower_col']
            
            if lower_band_col in row:
                bbands_condition = row['close'] < row[lower_band_col]
                conditions_met.append(bbands_condition)
                if bbands_condition:
                    print('BBands met at', row['close'], ' ', row[lower_band_col])
    
    # Check if we have any conditions to evaluate
    if conditions_total == 0:
        print("Warning: No conditions were configured to check")
        return False
    
    if len(conditions_met) == 0:
        print("Warning: No conditions could be evaluated (missing data?)")
        return False
    
    # All conditions must be TRUE for a buy signal
    all_conditions_met = all(conditions_met)
    print(f"Overall buy signal: {'YES' if all_conditions_met else 'NO'} ({sum(conditions_met)}/{len(conditions_met)} conditions met)")
    
    return all_conditions_met

# Main simulation function
def run_strategy_simulation(symbol, start_date, end_date, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings):
    # Fetch historical data
    df = get_historical_data(symbol, start=start_date, end=end_date)
    
    if df.empty:
        print("No data available. Exiting simulation.")
        return
    
    # Calculate indicators
    df = calculate_indicators(df, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings)
    
    # Print column names for debugging    
    # Track trades
    trades = []
    
    # Run simulation
    for i in range(len(df)):
        # Get current row as a Series
        current_time = df.index[i]
        row_series = df.iloc[i]
        
        print(f"\n--- Processing time: {current_time} ---")
        
        # Check conditions with the Series (not a DataFrame)
        if check_buy_conditions_for_row(row_series, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings):
            print(f"BUY SIGNAL at {current_time}, price: ${row_series['close']:.2f}")
            trades.append({
                'time': current_time,
                'action': 'BUY',
                'price': row_series['close']
            })
            place_order("buy")
        # Place buy order here if conditions are met   
        
        time.sleep(0.5)

             
    # Print trade summary
    if trades:
        print("\n=== TRADE SUMMARY ===")
        for trade in trades:
            print(f"{trade['time']} - {trade['action']} at ${trade['price']:.2f}")
    else:
        print("\nNo trades executed during the simulation period.")




In [16]:
run_strategy_simulation(
    symbol="SPY",
    start_date='2020-01-08', 
    end_date='2020-01-09',
    sma_settings=sma_settings,
    rsi_settings=rsi_settings,
    stoch_settings=stoch_settings,
    smi_settings=smi_settings,
    macd_settings=macd_settings,
    bbands_settings=bbands_settings
)


--- Processing time: 2020-01-08 00:00:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:01:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:02:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:03:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:04:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:05:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:06:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:07:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:08:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:09:00+00:00 ---
Overall buy signal: NO (0/2 conditions met)

--- Processing time: 2020-01-08 00:10:0

KeyboardInterrupt: 

In [ ]:
# Fetch historical data
start_date = '2020-01-08'
end_date = '2020-01-09'

df = get_historical_data(SYMBOL, start=start_date, end=end_date)
df = calculate_indicators(df, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings)

# Print column names to verify your indicators
print("DataFrame columns:", df.columns.tolist())

# Simulate real-time data updates
if not df.empty:
    for i in range(len(df)):
        # Get the current row (time point)
        current_row = df.iloc[i:i+1].copy()  # Using i:i+1 to keep it as a DataFrame
        
        # Debug: Print indicator values for this row
        if macd_settings:
            fast, slow, signal = macd_settings
            macd_col = f"MACD_{fast}_{slow}_{signal}"
            signal_col = f"MACDs_{fast}_{slow}_{signal}"
            if macd_col in current_row.columns and signal_col in current_row.columns:
                print(f"Time: {current_row.index[0]}, MACD: {current_row[macd_col].values[0]:.4f}, Signal: {current_row[signal_col].values[0]:.4f}")
        
        # Check if we have an existing position
        position = api.get_position(SYMBOL) if SYMBOL in [pos.symbol for pos in api.list_positions()] else None
        
        # Check conditions on just this row
        if check_buy_conditions_for_row(current_row, sma_settings, rsi_settings, stoch_settings, smi_settings, macd_settings, bbands_settings):
            print(f"Buy signal triggered at {current_row.index[0]}")
            place_order("buy")
        
        time.sleep()  # Simulate waiting for the next minute

DataFrame columns: ['close', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0']
Time: 2020-01-08 00:00:00+00:00, MACD: nan, Signal: nan


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().